#Download library

In [ ]:
!pip install tensorflow_ranking

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.4/150.4 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 35.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4


#Import libraries

In [ ]:
import datetime, os
import tensorflow as tf
import numpy as np
from pathlib import Path
from matplotlib import pyplot as plt
from tensorflow.keras import layers
from tensorflow import keras
from tensorflow.keras.preprocessing import image_dataset_from_directory
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import sparse_categorical_accuracy
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization
import glob
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.metrics import categorical_accuracy, top_k_categorical_accuracy, categorical_crossentropy
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras.models import Model

import datetime, os
import tensorflow as tf
from tensorflow import keras
import numpy as np
from pathlib import Path
from matplotlib import pyplot as plt
#from tensorflow.keras import layers
#import tensorflow_addons as tfa
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras import layers as L
from keras import regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import sparse_categorical_accuracy
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization
import glob
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.metrics import categorical_accuracy, top_k_categorical_accuracy, categorical_crossentropy
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import tensorflow_ranking as tfr
from keras.applications import imagenet_utils

In [ ]:
def apk(actual, predicted, k=5):
    if len(predicted) > k:
        predicted = predicted[:k]
    score = 0.0
    num_hits = 0.0
    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i + 1.0)
    if not actual:
        return 0.0
    return score / min(len(actual), k)

def mapk(actual, predicted, k=5):
    return np.mean([apk(a, p, k) for a, p in zip(actual, predicted)])


def top_5_accuracy(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=5)

In [ ]:
class Lion(tf.keras.optimizers.legacy.Optimizer):

  def __init__(self,
               learning_rate=0.0001,
               beta_1=0.9,
               beta_2=0.99,
               wd=0,
               name='lion',
               **kwargs):

    super(Lion, self).__init__(name, **kwargs)
    self._set_hyper('learning_rate', kwargs.get('lr', learning_rate))
    self._set_hyper('beta_1', beta_1)
    self._set_hyper('beta_2', beta_2)
    self._set_hyper('wd', wd)

  def _create_slots(self, var_list):
    # Create slots for the first and second moments.
    # Separate for-loops to respect the ordering of slot variables from v1.
    for var in var_list:
      self.add_slot(var, 'm')

  def _prepare_local(self, var_device, var_dtype, apply_state):
    super(Lion, self)._prepare_local(var_device, var_dtype, apply_state)

    beta_1_t = tf.identity(self._get_hyper('beta_1', var_dtype))
    beta_2_t = tf.identity(self._get_hyper('beta_2', var_dtype))
    wd_t = tf.identity(self._get_hyper('wd', var_dtype))
    lr = apply_state[(var_device, var_dtype)]['lr_t']
    apply_state[(var_device, var_dtype)].update(
        dict(
            lr=lr,
            beta_1_t=beta_1_t,
            one_minus_beta_1_t=1 - beta_1_t,
            beta_2_t=beta_2_t,
            one_minus_beta_2_t=1 - beta_2_t,
            wd_t=wd_t))

  @tf.function(jit_compile=True)
  def _resource_apply_dense(self, grad, var, apply_state=None):
    var_device, var_dtype = var.device, var.dtype.base_dtype
    coefficients = ((apply_state or {}).get((var_device, var_dtype)) or
                    self._fallback_apply_state(var_device, var_dtype))

    m = self.get_slot(var, 'm')
    var_t = var.assign_sub(
        coefficients['lr_t'] *
        (tf.math.sign(m * coefficients['beta_1_t'] +
                      grad * coefficients['one_minus_beta_1_t']) +
         var * coefficients['wd_t']))
    with tf.control_dependencies([var_t]):
      m.assign(m * coefficients['beta_2_t'] +
               grad * coefficients['one_minus_beta_2_t'])

  @tf.function(jit_compile=True)
  def _resource_apply_sparse(self, grad, var, indices, apply_state=None):
    var_device, var_dtype = var.device, var.dtype.base_dtype
    coefficients = ((apply_state or {}).get((var_device, var_dtype)) or
                    self._fallback_apply_state(var_device, var_dtype))

    m = self.get_slot(var, 'm')
    m_t = m.assign(m * coefficients['beta_1_t'])
    m_scaled_g_values = grad * coefficients['one_minus_beta_1_t']
    m_t = m_t.scatter_add(tf.IndexedSlices(m_scaled_g_values, indices))
    var_t = var.assign_sub(coefficients['lr'] *
                           (tf.math.sign(m_t) + var * coefficients['wd_t']))

    with tf.control_dependencies([var_t]):
      m_t = m_t.scatter_add(tf.IndexedSlices(-m_scaled_g_values, indices))
      m_t = m_t.assign(m_t * coefficients['beta_2_t'] /
                       coefficients['beta_1_t'])
      m_scaled_g_values = grad * coefficients['one_minus_beta_2_t']
      m_t.scatter_add(tf.IndexedSlices(m_scaled_g_values, indices))

  def get_config(self):
    config = super(Lion, self).get_config()
    config.update({
        'learning_rate': self._serialize_hyperparameter('learning_rate'),
        'beta_1': self._serialize_hyperparameter('beta_1'),
        'beta_2': self._serialize_hyperparameter('beta_2'),
        'wd': self._serialize_hyperparameter('wd'),
    })
    return config

#Load model

In [ ]:
model = tf.keras.models.load_model('/content/IRB_model.h5',custom_objects={"top_5_accuracy": top_5_accuracy,"Lion":Lion })
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 28, 28, 1)]  0           []                               
                                                                                                  
 rescaling_5 (Rescaling)        (None, 28, 28, 1)    0           ['input_6[0][0]']                
                                                                                                  
 batch_normalization_106 (Batch  (None, 28, 28, 1)   4           ['rescaling_5[0][0]']            
 Normalization)                                                                                   
                                                                                                  
 conv2d_70 (Conv2D)             (None, 28, 28, 64)   640         ['batch_normalization_106[0

#Visualization feature map

In [ ]:
from matplotlib import pyplot

In [ ]:
ixs =[]

In [ ]:
for i in range(len(model.layers)):
 layer = model.layers[i]
 # check for convolutional layer
 if 'conv2d' not in layer.name:
   continue
 # summarize output shape
 ixs.append(i)
 print(i, layer.name, layer.output.shape)

3 conv2d_70 (None, 28, 28, 64)
9 conv2d_71 (None, 28, 28, 256)
15 depthwise_conv2d_31 (None, 28, 28, 256)
21 conv2d_72 (None, 28, 28, 64)
24 conv2d_73 (None, 28, 28, 256)
31 depthwise_conv2d_32 (None, 14, 14, 256)
37 conv2d_74 (None, 14, 14, 96)
39 conv2d_75 (None, 14, 14, 384)
45 depthwise_conv2d_33 (None, 14, 14, 384)
51 conv2d_76 (None, 14, 14, 96)
54 conv2d_77 (None, 14, 14, 384)
60 depthwise_conv2d_34 (None, 14, 14, 384)
66 conv2d_78 (None, 14, 14, 96)
70 conv2d_79 (None, 14, 14, 384)
77 depthwise_conv2d_35 (None, 7, 7, 384)
83 conv2d_80 (None, 7, 7, 128)
85 conv2d_81 (None, 7, 7, 512)
91 depthwise_conv2d_36 (None, 7, 7, 512)
97 conv2d_82 (None, 7, 7, 128)
100 conv2d_83 (None, 7, 7, 512)
106 depthwise_conv2d_37 (None, 7, 7, 512)
112 conv2d_84 (None, 7, 7, 128)
116 conv2d_85 (None, 7, 7, 512)
123 depthwise_conv2d_38 (None, 4, 4, 512)
129 conv2d_86 (None, 4, 4, 256)
131 conv2d_87 (None, 4, 4, 1024)
137 depthwise_conv2d_39 (None, 4, 4, 1024)
143 conv2d_88 (None, 4, 4, 256)
146 conv2d

In [ ]:
outputs = [model.layers[i].output for i in ixs]
models = Model(inputs=model.inputs, outputs=outputs)
# load the image with the required shape
img = cv2.imread('/content/4506595474014208.png',0)
img = img.reshape(28,28,1)
# expand dimensions so that it represents a single 'sample'
img = np.expand_dims(img, axis=0)
# prepare the image (e.g. scale pixel values for the vgg)
# get feature map for first hidden layer
feature_maps = models.predict(img)
print()
# plot the output from each block
square = 8
for fmap in feature_maps:
	# plot all 64 maps in an 8x8 squares
	ix = 1
	for _ in range(square):
		for _ in range(square):
			# specify subplot and turn of axis
			ax = pyplot.subplot(square, square, ix)
			ax.set_xticks([])
			ax.set_yticks([])
			# plot filter channel in grayscale
			pyplot.imshow(fmap[0, :, :, ix-1], cmap='gray')
			ix += 1
	# show the figure
	pyplot.show()
	print()

#Visualization all layers

In [ ]:
layer_outputs = [layer.output for layer in model.layers[1:]]
#print(layer_outputs)
visualize_model = tf.keras.models.Model(inputs = model.input, outputs = layer_outputs)
img = cv2.imread('/content/4506595474014208.png',0)
img = img.reshape(28,28,1)
# expand dimensions so that it represents a single 'sample'
img = np.expand_dims(img, axis=0)
feature_maps=visualize_model.predict(img)
print(len(feature_maps))

1/1 [==============================] - 2s 2s/step
185


In [ ]:
# Show names of layers available in model
layer_names = [layer.name for layer in model.layers]
print(layer_names)

['input_6', 'rescaling_5', 'batch_normalization_106', 'conv2d_70', 'batch_normalization_107', 'tf.convert_to_tensor_69', 'tf.math.softplus_69', 'tf.math.tanh_69', 'tf.math.multiply_69', 'conv2d_71', 'batch_normalization_108', 'tf.convert_to_tensor_70', 'tf.math.softplus_70', 'tf.math.tanh_70', 'tf.math.multiply_70', 'depthwise_conv2d_31', 'batch_normalization_109', 'tf.convert_to_tensor_71', 'tf.math.softplus_71', 'tf.math.tanh_71', 'tf.math.multiply_71', 'conv2d_72', 'batch_normalization_110', 'add_22', 'conv2d_73', 'batch_normalization_111', 'tf.convert_to_tensor_72', 'tf.math.softplus_72', 'tf.math.tanh_72', 'tf.math.multiply_72', 'zero_padding2d_9', 'depthwise_conv2d_32', 'batch_normalization_112', 'tf.convert_to_tensor_73', 'tf.math.softplus_73', 'tf.math.tanh_73', 'tf.math.multiply_73', 'conv2d_74', 'batch_normalization_113', 'conv2d_75', 'batch_normalization_114', 'tf.convert_to_tensor_74', 'tf.math.softplus_74', 'tf.math.tanh_74', 'tf.math.multiply_74', 'depthwise_conv2d_33', '

In [ ]:
# import required libraries
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Plotting the graph
for layer_names, feature_maps in zip(layer_names,feature_maps):
  print(feature_maps.shape)
  if len(feature_maps.shape) == 4 :
    channels = feature_maps.shape[-1]
    size = feature_maps.shape[1]
    display_grid = np.zeros((size, size * channels))
    for i in range(channels):
      x = feature_maps[0, :, :, i]
      x -= x.mean()
      x /= x.std()
      x *= 64
      x += 128
      x = np.clip(x, 0, 255).astype('uint8')
      # We'll tile each filter into this big horizontal grid
      display_grid[:, i * size : (i + 1) * size] = x
    scale = 20. / channels
    plt.figure(figsize=(scale * channels, scale))
    plt.title(layer_names)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='viridis')